<a href="https://colab.research.google.com/github/osapii/quickstart-ios/blob/master/Getting_started_with_BigQuery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bigqueryの実行

マジックコマンドを使用する事で、colab上からBigQueryのクエリを実行する事ができる

In [143]:
%%bigquery --project voicy-aa935 dataframe

with playlist_summary as (select s.*
,t.name
from
(select speaker_id
,count(distinct playlist_id) as playlist_num
,avg(chapter_num) as chapter_num
,avg(playlist_length) as playlist_duration
,cast(round(avg(chapter_num),0) as int64) as chapter_num_round
,cast(round(avg(playlist_length),0) as int64) as playlist_duration_round
from 
(select speaker_id
,playlist_id
,count(*) as chapter_num
,round(sum(duration)/1000/60,1) as playlist_length
,max(created) as created
from `voicy_rds.t_voice`
group by speaker_id,playlist_id
having cast(created as string) like "%%2018-10%%"
) as s
group by speaker_id
) as s
left join
voicy_rds.t_speaker as t
on
s.speaker_id = t.id
)

select s.*
,t.uu_rank
,t.rank_range
from
playlist_summary as s
left join
(select speaker_id
,count(distinct uuid) as total_play_uu
,rank() over (order by count(distinct uuid) desc) as uu_rank
,case when rank() over (order by count(distinct uuid) desc) < 31 then "top30" else "over30" end as rank_range 
from `voicy_player_play.play_201810*` 
where platform = "ios"
group by speaker_id) as t
on
s.speaker_id  = t.speaker_id
where rank_range is not null

,speaker_id,playlist_num,chapter_num,playlist_duration,chapter_num_round,playlist_duration_round,name,uu_rank,rank_range
0,1122,1,3.000000,12.300000,3,12,児玉 健（こだま たけし）,132,over30
1,565,1,3.000000,5.500000,3,6,Kozue,180,over30
2,2452,1,4.000000,1.600000,4,2,DIO,229,over30
3,1945,1,4.000000,12.400000,4,12,財部亮治(たからべりょうじ),106,over30
4,686,1,4.000000,11.000000,4,11,如月サラ,189,over30
5,470,1,5.000000,22.400000,5,22,大貫 由佳,189,over30
6,1900,1,5.000000,9.400000,5,9,momo,85,over30
7,1243,1,5.000000,27.000000,5,27,onomiyuki,162,over30
8,1601,1,5.000000,36.300000,5,36,クーニン,209,over30
9,114,1,5.000000,15.400000,5,15,Qoomin,143,over30


In [141]:
import altair as alt
from vega_datasets import data

chart0 = alt.Chart(dataframe).mark_point().encode(
    x=alt.X('chapter_num:Q',title='1プレイリストあたりのチャプター数'),
    y=alt.Y('playlist_duration:Q',title='1プレイリストの長さ(分)'),
    color='rank_range:N',
    tooltip=['name:N','chapter_num:N','playlist_duration:N'],
    text = 'name'
).properties(title='チャプター数 x 1プレイリストの長さ (TOP30とそれ以外で比較)')

chart1 = alt.Chart(dataframe).mark_bar().encode(
    x=alt.X('chapter_num_round:Q',title = '1プレイリストあたりのチャプター数'),
    y=alt.Y('count(chapter_num_round):Q',title='パーソナリティ数'),
    tooltip = ['count(chapter_num_round)','chapter_num_round']
).properties(title='平均チャプター数xパーソナリティ数')


chart2 = alt.Chart(dataframe).mark_bar().encode(
    x=alt.X('playlist_duration_round:Q',title = '1プレイリストあたりの長さ(分)',bin=alt.Bin(maxbins=10)),
    y=alt.Y('count(playlist_duration_round):Q',title='パーソナリティ数')
).properties(title='平均プレイリストの長さ(分)xパーソナリティ数')

chart1 | chart2 | chart0

HConcatChart({
  hconcat: [Chart({
    data:      speaker_id  playlist_num  chapter_num  playlist_duration  \
    0          1122             1     3.000000          12.300000   
    1           565             1     3.000000           5.500000   
    2          2452             1     4.000000           1.600000   
    3          1945             1     4.000000          12.400000   
    4           686             1     4.000000          11.000000   
    5           470             1     5.000000          22.400000   
    6          1900             1     5.000000           9.400000   
    7          1243             1     5.000000          27.000000   
    8          1601             1     5.000000          36.300000   
    9           114             1     5.000000          15.400000   
    10         2445             1     6.000000          18.400000   
    11          605             1     6.000000          14.200000   
    12         1371             1     6.000000          45.200000   
    13          666             1     6.000000          16.100000   
    14          170             1     7.000000          50.700000   
    15         1331             1     7.000000          12.400000   
    16          836             1     7.000000          10.900000   
    17          240             1     7.000000          29.900000   
    18          794             1     8.000000           4.400000   
    19         2270             1     1.000000           5.500000   
    20         2070             1     1.000000           0.200000   
    21          832             2     2.500000          10.750000   
    22         1790             2     2.500000          13.300000   
    23          767             2     3.000000           8.400000   
    24         2194             2     3.500000           9.400000   
    25          697             2     4.500000           8.700000   
    26          388             2     4.500000           8.550000   
    27          672             2     5.000000           5.550000   
    28         1898             2     5.000000          39.400000   
    29          676             2     5.000000           4.400000   
    ..          ...           ...          ...                ...   
    129         659            23    13.913043          10.521739   
    130         560            24     2.000000           5.666667   
    131        1086            24     4.000000          11.245833   
    132         855            24    14.083333          11.537500   
    133        1090            25     3.880000          10.168000   
    134         851            25     4.600000           9.920000   
    135        1719            26     4.076923          18.480769   
    136        1743            29     5.103448          34.517241   
    137        2332            30     1.000000           4.966667   
    138        1135            30     3.700000          17.343333   
    139        1173            30     5.000000          11.326667   
    140        1726            31     4.000000          18.858065   
    141        1343            31     8.000000           5.112903   
    142        1344            31     8.000000           6.990323   
    143         514            31    10.000000          16.009677   
    144         481            31     6.258065          38.387097   
    145        1133            31     4.580645          13.383871   
    146        2244            31     3.806452          15.709677   
    147         607            31     6.870968          14.906452   
    148        1588            31     4.419355          11.309677   
    149         708            31     3.967742          10.516129   
    150        1860            31     3.096774          15.509677   
    151        1162            32     3.500000          19.928125   
    152        1488            32     4.062500           4.334375   
    153         533            32     9.687500          14.896875   
    154        2390       



fdsf
